In [30]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [3]:
mx.context.num_gpus()

0

In [4]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/pollution.csv')

In [5]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [6]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'date'], infer_datetime_format=True)

In [7]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0,2010-01-02 00:00:00
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0,2010-01-02 01:00:00
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0,2010-01-02 02:00:00
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0,2010-01-02 03:00:00
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [8]:
df.columns

Index(['date', 'pollution', 'dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain',
       'NE', 'NW', 'SE', 'cv', 'timestamp'],
      dtype='object')

In [197]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [198]:
train_df.shape

(35040, 13)

In [199]:
test_df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
35040,2014-01-01 00:00:00,24.0,-20,7.0,1014.0,143.48,0,0,0,1,0,0,2014-01-01 00:00:00
35041,2014-01-01 01:00:00,53.0,-20,7.0,1013.0,147.50,0,0,0,1,0,0,2014-01-01 01:00:00
35042,2014-01-01 02:00:00,65.0,-20,6.0,1013.0,151.52,0,0,0,1,0,0,2014-01-01 02:00:00
35043,2014-01-01 03:00:00,70.0,-20,6.0,1013.0,153.31,0,0,0,1,0,0,2014-01-01 03:00:00
35044,2014-01-01 04:00:00,79.0,-18,3.0,1012.0,0.89,0,0,0,0,0,1,2014-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [200]:
train_ds = PandasDataset(
    train_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

In [201]:
test_ds = PandasDataset(
    test_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

#### training

In [202]:
K = 24
n_hours = 24*3

In [203]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=200,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06,)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=64,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
#     time_features=['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv'],
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:03<00:00,  1.83s/it, epoch=76/200, avg_epoch_loss=7.73]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:02<00:00,  1.83s/it, epoch=152/200, avg_epoch_loss=6.51]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:59<00:00,  1.80s/it, epoch=200/200, avg_epoch_loss=5.72]


#### predicted y

In [186]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [187]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[100.81339264  87.56984711  73.31034851  62.21865845  53.8684845
  48.23006821  45.01316833  43.30499649  42.38579178  41.96973038
  42.00999832  42.59859848]
(12,)
------
1
[100.81339264  87.56984711  73.31034851  62.21865845  53.8684845
  48.23006821  45.01316833  43.30499649  42.38579178  41.96973038
  42.00999832  42.59859848  77.71930695  69.36290741  60.48844528
  52.07729721  45.20361328  40.55935669  38.06038284  36.66895676
  35.86222076  35.54566574  35.88199997  37.24533463]
(24,)
------
2
[100.81339264  87.56984711  73.31034851  62.21865845  53.8684845
  48.23006821  45.01316833  43.30499649  42.38579178  41.96973038
  42.00999832  42.59859848  77.71930695  69.36290741  60.48844528
  52.07729721  45.20361328  40.55935669  38.06038284  36.66895676
  35.86222076  35.54566574  35.88199997  37.24533463  79.9138031
  70.70348358  60.0987854   48.5494957   38.1125412   31.15009499
  26.93515015  24.41718864  23.32116508  23.71865654  25.75067139
  29.27259254]
(36,)
------
3
[1

[100.81339264  87.56984711  73.31034851  62.21865845  53.8684845
  48.23006821  45.01316833  43.30499649  42.38579178  41.96973038
  42.00999832  42.59859848  77.71930695  69.36290741  60.48844528
  52.07729721  45.20361328  40.55935669  38.06038284  36.66895676
  35.86222076  35.54566574  35.88199997  37.24533463  79.9138031
  70.70348358  60.0987854   48.5494957   38.1125412   31.15009499
  26.93515015  24.41718864  23.32116508  23.71865654  25.75067139
  29.27259254  84.70713806  74.7912674   63.41917801  51.35997009
  40.34706879  33.13939285  29.10622215  26.94266701  26.13335609
  26.57991982  28.42135048  31.81804085  67.57558441  59.95028687
  53.44702148  47.32152176  41.26984787  36.10997391  32.09783936
  29.22081184  27.61329651  27.86387634  31.11777306  37.5610733
  73.41695404  68.88560486  63.71976852  58.45289993  53.20325851
  48.71632004  45.42315674  43.71841049  44.40400314  48.39547729
  54.93371582  63.07117844  75.05483246  73.24359131  70.5866394
  67.22424316 

[100.81339264  87.56984711  73.31034851  62.21865845  53.8684845
  48.23006821  45.01316833  43.30499649  42.38579178  41.96973038
  42.00999832  42.59859848  77.71930695  69.36290741  60.48844528
  52.07729721  45.20361328  40.55935669  38.06038284  36.66895676
  35.86222076  35.54566574  35.88199997  37.24533463  79.9138031
  70.70348358  60.0987854   48.5494957   38.1125412   31.15009499
  26.93515015  24.41718864  23.32116508  23.71865654  25.75067139
  29.27259254  84.70713806  74.7912674   63.41917801  51.35997009
  40.34706879  33.13939285  29.10622215  26.94266701  26.13335609
  26.57991982  28.42135048  31.81804085  67.57558441  59.95028687
  53.44702148  47.32152176  41.26984787  36.10997391  32.09783936
  29.22081184  27.61329651  27.86387634  31.11777306  37.5610733
  73.41695404  68.88560486  63.71976852  58.45289993  53.20325851
  48.71632004  45.42315674  43.71841049  44.40400314  48.39547729
  54.93371582  63.07117844  75.05483246  73.24359131  70.5866394
  67.22424316 

[100.81339264  87.56984711  73.31034851  62.21865845  53.8684845
  48.23006821  45.01316833  43.30499649  42.38579178  41.96973038
  42.00999832  42.59859848  77.71930695  69.36290741  60.48844528
  52.07729721  45.20361328  40.55935669  38.06038284  36.66895676
  35.86222076  35.54566574  35.88199997  37.24533463  79.9138031
  70.70348358  60.0987854   48.5494957   38.1125412   31.15009499
  26.93515015  24.41718864  23.32116508  23.71865654  25.75067139
  29.27259254  84.70713806  74.7912674   63.41917801  51.35997009
  40.34706879  33.13939285  29.10622215  26.94266701  26.13335609
  26.57991982  28.42135048  31.81804085  67.57558441  59.95028687
  53.44702148  47.32152176  41.26984787  36.10997391  32.09783936
  29.22081184  27.61329651  27.86387634  31.11777306  37.5610733
  73.41695404  68.88560486  63.71976852  58.45289993  53.20325851
  48.71632004  45.42315674  43.71841049  44.40400314  48.39547729
  54.93371582  63.07117844  75.05483246  73.24359131  70.5866394
  67.22424316 

[100.81339264  87.56984711  73.31034851  62.21865845  53.8684845
  48.23006821  45.01316833  43.30499649  42.38579178  41.96973038
  42.00999832  42.59859848  77.71930695  69.36290741  60.48844528
  52.07729721  45.20361328  40.55935669  38.06038284  36.66895676
  35.86222076  35.54566574  35.88199997  37.24533463  79.9138031
  70.70348358  60.0987854   48.5494957   38.1125412   31.15009499
  26.93515015  24.41718864  23.32116508  23.71865654  25.75067139
  29.27259254  84.70713806  74.7912674   63.41917801  51.35997009
  40.34706879  33.13939285  29.10622215  26.94266701  26.13335609
  26.57991982  28.42135048  31.81804085  67.57558441  59.95028687
  53.44702148  47.32152176  41.26984787  36.10997391  32.09783936
  29.22081184  27.61329651  27.86387634  31.11777306  37.5610733
  73.41695404  68.88560486  63.71976852  58.45289993  53.20325851
  48.71632004  45.42315674  43.71841049  44.40400314  48.39547729
  54.93371582  63.07117844  75.05483246  73.24359131  70.5866394
  67.22424316 

...

[100.81339264  87.56984711  73.31034851 ...  71.55005646  65.06835175
  59.17607117]
(99336,)
------
8278
[100.81339264  87.56984711  73.31034851 ...  54.80792618  51.39995193
  48.29361343]
(99348,)
------
8279
[100.81339264  87.56984711  73.31034851 ...  66.8143692   62.26127243
  58.71490097]
(99360,)
------
8280
[100.81339264  87.56984711  73.31034851 ... 102.6825943  102.27449799
 105.04686737]
(99372,)
------
8281
[100.81339264  87.56984711  73.31034851 ...  88.3297348   86.4212265
  88.06528473]
(99384,)
------
8282
[100.81339264  87.56984711  73.31034851 ... 179.74760437 186.23176575
 195.48457336]
(99396,)
------
8283
[100.81339264  87.56984711  73.31034851 ... 181.46333313 192.00561523
 205.68774414]
(99408,)
------
8284
[100.81339264  87.56984711  73.31034851 ... 213.56825256 229.00927734
 244.08491516]
(99420,)
------
8285
[100.81339264  87.56984711  73.31034851 ... 247.23687744 260.5173645
 277.60507202]
(99432,)
------
8286
[100.81339264  87.56984711  73.31034851 ... 268.

[100.81339264  87.56984711  73.31034851 ... 152.85292053 162.81655884
 175.8993988 ]
(100320,)
------
8360
[100.81339264  87.56984711  73.31034851 ... 190.10505676 200.95161438
 213.64489746]
(100332,)
------
8361
[100.81339264  87.56984711  73.31034851 ... 175.42462158 187.67626953
 197.91705322]
(100344,)
------
8362
[100.81339264  87.56984711  73.31034851 ... 185.17829895 194.22361755
 202.34033203]
(100356,)
------
8363
[100.81339264  87.56984711  73.31034851 ... 153.70252991 156.56040955
 158.42604065]
(100368,)
------
8364
[100.81339264  87.56984711  73.31034851 ... 159.60586548 161.49064636
 162.57019043]
(100380,)
------
8365
[100.81339264  87.56984711  73.31034851 ... 159.87878418 162.49681091
 164.08131409]
(100392,)
------
8366
[100.81339264  87.56984711  73.31034851 ... 183.1136322  187.25852966
 191.26277161]
(100404,)
------
8367
[100.81339264  87.56984711  73.31034851 ... 176.07597351 177.39543152
 177.51780701]
(100416,)
------
8368
[100.81339264  87.56984711  73.310348

[100.81339264  87.56984711  73.31034851 ...  39.94425964  38.53340149
  36.67614365]
(101328,)
------
8444
[100.81339264  87.56984711  73.31034851 ...  49.97808456  48.37183762
  46.89326096]
(101340,)
------
8445
[100.81339264  87.56984711  73.31034851 ...  42.53847122  39.29453659
  35.63232803]
(101352,)
------
8446
[100.81339264  87.56984711  73.31034851 ...  50.42255783  48.66204453
  47.1162529 ]
(101364,)
------
8447
[100.81339264  87.56984711  73.31034851 ...  51.50379562  50.98422623
  50.92102432]
(101376,)
------
8448
[100.81339264  87.56984711  73.31034851 ...  53.51513672  54.54790115
  56.22984695]
(101388,)
------
8449
[100.81339264  87.56984711  73.31034851 ...   0.88258362  -1.96239471
  -3.93068814]
(101400,)
------
8450
[100.81339264  87.56984711  73.31034851 ...  58.25037003  61.25305176
  63.8949585 ]
(101412,)
------
8451
[100.81339264  87.56984711  73.31034851 ...  59.23496628  61.10538864
  62.77318192]
(101424,)
------
8452
[100.81339264  87.56984711  73.310348

[100.81339264  87.56984711  73.31034851 ...  15.19004059  15.1774292
  15.37437439]
(102276,)
------
8523
[100.81339264  87.56984711  73.31034851 ...   6.9297328    6.81315088
   7.25802088]
(102288,)
------
8524
[100.81339264  87.56984711  73.31034851 ...   3.73017025   4.22706175
   5.43090963]
(102300,)
------
8525
[100.81339264  87.56984711  73.31034851 ...   8.41864204   8.29294968
   9.5257597 ]
(102312,)
------
8526
[100.81339264  87.56984711  73.31034851 ...   9.85826778  10.54547215
  12.13210392]
(102324,)
------
8527
[100.81339264  87.56984711  73.31034851 ...  13.76195908  14.41713047
  16.97788811]
(102336,)
------
8528
[100.81339264  87.56984711  73.31034851 ...  11.26086521  12.80072308
  14.31434917]
(102348,)
------
8529
[100.81339264  87.56984711  73.31034851 ...  17.25309944  19.73364449
  19.4508152 ]
(102360,)
------
8530
[100.81339264  87.56984711  73.31034851 ...  40.89289474  40.56541443
  40.25932693]
(102372,)
------
8531
[100.81339264  87.56984711  73.3103485

[100.81339264  87.56984711  73.31034851 ... 375.37069702 413.72647095
 418.07263184]
(103272,)
------
8606
[100.81339264  87.56984711  73.31034851 ... 174.09649658 167.81350708
 157.09593201]
(103284,)
------
8607
[100.81339264  87.56984711  73.31034851 ...  92.88700104  85.46508789
  81.27605438]
(103296,)
------
8608
[100.81339264  87.56984711  73.31034851 ... 231.36402893 219.0867157
 205.09022522]
(103308,)
------
8609
[100.81339264  87.56984711  73.31034851 ... 251.58558655 231.47517395
 209.86598206]
(103320,)
------
8610
[100.81339264  87.56984711  73.31034851 ... 101.83159637  98.01538086
  94.97505951]
(103332,)
------
8611
[100.81339264  87.56984711  73.31034851 ...  58.56315613  58.43379593
  58.64125061]
(103344,)
------
8612
[100.81339264  87.56984711  73.31034851 ... 145.15974426 126.6969986
 111.92572784]
(103356,)
------
8613
[100.81339264  87.56984711  73.31034851 ... 168.06492615 152.59263611
 140.05345154]
(103368,)
------
8614
[100.81339264  87.56984711  73.31034851

In [188]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(8677, 12)

In [189]:
test_ds_predict_q50_supervised[0:3]

array([[100.81339264,  87.56984711,  73.31034851,  62.21865845,
         53.8684845 ,  48.23006821,  45.01316833,  43.30499649,
         42.38579178,  41.96973038,  42.00999832,  42.59859848],
       [ 77.71930695,  69.36290741,  60.48844528,  52.07729721,
         45.20361328,  40.55935669,  38.06038284,  36.66895676,
         35.86222076,  35.54566574,  35.88199997,  37.24533463],
       [ 79.9138031 ,  70.70348358,  60.0987854 ,  48.5494957 ,
         38.1125412 ,  31.15009499,  26.93515015,  24.41718864,
         23.32116508,  23.71865654,  25.75067139,  29.27259254]])

#### actual y

In [190]:
from helper import series_to_supervised, stage_series_to_supervised

In [191]:
test_df_supervised = series_to_supervised(test_df[['pollution']], n_hours, K)

In [192]:
test_df_supervised.iloc[:, -K:].shape

(8677, 12)

In [193]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [194]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),var1(t+10),var1(t+11)
35112,85.0,86.0,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0
35113,86.0,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0
35114,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0
35115,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0,176.0
35116,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0,176.0,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...
43784,8.0,8.0,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0
43785,8.0,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0
43786,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0,10.0
43787,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0,10.0,8.0


#### performance on test set

In [195]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 37.236
RMSE = 61.636
